# Preprocess subject data

In [1]:
import sys
import os
from pathlib import Path
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject, Group
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 


imported src module
imported preprocessing module
imported configs module
imported utils module


Load in subject IDs and paths from CFGLog

In [2]:
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC
Panc_subject_ids = config.data.chronic_pancreatitis.subject_ids.CP
Lupus_CP_subject_ids = config.data.lupus.subject_ids.CP
Lupus_NP_subject_ids = config.data.lupus.subject_ids.NP

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

Define preprocessing parameters

In [3]:
TIME_RANGE = (-2.5, 2.5)
PERISTIM_TIME_WIN = abs(TIME_RANGE[0]) + abs(TIME_RANGE[1])
# BASELINE = (-0.2, 0)

In [4]:
times_tup, time_win_path = pre_utils.get_time_window(PERISTIM_TIME_WIN)

[-2.5,0.0,2.5]


Combine subject IDs

In [5]:
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids,
    "chronic_pancreatitis": Panc_subject_ids,
    "lupus": Lupus_CP_subject_ids + Lupus_NP_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain']) + len(sub_ids['chronic_pancreatitis']) + len(sub_ids['lupus'])}")
print(sub_ids)

Total subjects: 64
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.'], 'chronic_pancreatitis': ['002', '003', '006', '007', '008', '009', '010', '011', '012', '013', '014'], 'lupus': ['5186', '6310', '5295', '5873', '6100', '6106', '5648', '5675', '5845', '5713']}


Create Subject object for each subject

In [ ]:
# MAKE SURE TO COMMENT ONCE THIS IS RUN ONCE.
# IF FORGET, CHECK FOR THE NOISY SPAN IN SIGVIEWER
sub_ids = {
    "033": [(246.5, 248)],
    "040": [(0.0, 11.0)],
    "C19": [(15.0, 16)],
    "C16": [
        (275.5, 277.5),
        (280.4, 280.8),
        (281.2, 281.8),
    ],
    "C15": [(249.5, 251.25)],
    "C13": [
        (2.0, 3.5),
        (14.25, 16.0),
        (18.5, 19.5),
        (142, 143),
        (146.5, 148),
        (255.5, 260),
    ],
    "C12": [(113.5, 115.5)],
    "C11": [
        (56.3, 58),
        (62, 62.5),
        (114.7, 117),
        (155.2, 156),
        (157.5, 159),
        (241.5, 243.7),
    ],
    "C9.": [
        (5.3, 7.5),
        (40, 42.8),
        (132, 136),
        (184.5, 189),
        (261.2, 265.5),
        (293.5, 295),
    ],
    "050": [(58.2, 59.5), (203.7, 206.3)],
    "C7.": [
        (88.5, 90.5),
        (272, 274),
    ],
    "C6.": [(70.2, 75.8)],
    "C2.": [
        (0, 1),
        (5, 8),
        (15.5, 16.5),
        (222.5, 223),
        (224, 224.5),
    ],
    "059": [(37.3, 38.5), (162.2, 163.6), (221.2, 223.5), (305,)],
}

# %%
check_done = True
check_done = False

# %%
for sub_id in sub_ids:
    if check_done and os.path.exists(resting_save_path / f"{sub_id}_eyes_open-raw.fif"):
        print(f"Subject {sub_id} already cropped")
        continue
    print(f"Subject {sub_id}: cropping noisy spans")

    # Load eyes open raw
    raw_eo = mne.io.read_raw_fif(processed_data_path / f"{sub_id}_eyes_open-raw.fif")

    # Keep track of total time cropped
    total_time_cropped = 0

    # Crop based on provided times
    # reverse_sub_ids = sub_ids[sub_id][::-1]
    reverse_sub_ids = sub_ids[sub_id]
    for i, crop_times in enumerate(reverse_sub_ids):
        print(
            f"Subject {sub_id}: cropping {i+1} of {len(sub_ids[sub_id])}: {crop_times}"
        )

        # Adjust crop_times by subtracting the total time cropped
        crop_times_adjusted = [
            crop_time - total_time_cropped for crop_time in crop_times
        ]
        crop_times = crop_times_adjusted

        # get crop times
        t1 = crop_times[0]
        t2 = crop_times[1] if len(crop_times) == 2 else None

        # if only one time is given use that as end time
        if t2 is None:
            raw_eo = raw_eo.crop(tmax=t1)
        else:
            raw_eo = pr.snip_span(raw_eo, t1, t2)

    # Save cropped raw
    raw_eo.export(resting_save_path / f"{sub_id}_eyes_open.vhdr", overwrite=True)
    raw_eo.save(resting_save_path / f"{sub_id}_eyes_open-raw.fif", overwrite=True)
    print(f"Subject {sub_id} noisy span cropped")

# %%

reverse_sub_ids


In [6]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        preprocessed_data_path = clbp_processed_path if group == "chronic_low_back_pain" else None
        subject = Subject(sub_id, group, preprocessed_data_path=preprocessed_data_path)
        print(subject.subject_folder)
        
        subject.preprocess()
        subject.get_cleaned_epochs(TIME_RANGE, PERISTIM_TIME_WIN)           
        break

        # subject.get_cleaned_resting()

        # subject.get_stc_resting()
        # subject.get_stc_epochs()


018.EEGDATA
Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/018.EEGDATA/Acquisition 018.Part2 Data.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2959599  =      0.000 ...  2959.599 secs...
Loaded raw for subject 018


AttributeError: 'tuple' object has no attribute 'ch_names'

In [ ]:
subject.preprocessed_raw